In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd

import unicodedata
import re
import json

import acquire

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from wordcloud import WordCloud

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filter="ignore"

from bs4 import BeautifulSoup
import prepare

In [2]:
def get_fifa_data():
    """ The original excel file can be found by going to the following website (https://www.kaggle.com/bryanb/fifa-player-stats-database)"""
    #looking for csv file
    filename = "fifa.csv"

    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # read the excel file dataframe, verify that they have a year column assigned.
        # import excel sheets frame from local file
        df = pd.read_excel('fifa_male.xlsx')
        # Read excel file
        xlsx = pd.ExcelFile('fifa_male.xlsx')
        # import data for year 2022
        df1 = pd.read_excel(xlsx, 'FIFA 22')
        # import data for year 2021
        df2 = pd.read_excel(xlsx, 'FIFA 21')
        # import data for year 2020
        df3 = pd.read_excel(xlsx, 'FIFA 20')
        # import data for year 2019
        df4 = pd.read_excel(xlsx, 'FIFA 19')
        # import data for year 2018
        df5 = pd.read_excel(xlsx, 'FIFA 18')
        # import data for year 2017
        df6 = pd.read_excel(xlsx, 'FIFA 17')
        # import data for year 2016
        df7 = pd.read_excel(xlsx, 'FIFA 16')
        # import data for year 2015
        df8 = pd.read_excel(xlsx, 'FIFA 15')
        "It is important that we add a year column so that our data is not contaminated across the years and to make exploration easier"
        # now to add a year column to each csv file
        df1['year'] = 2022
        df2['year'] = 2021
        df3['year'] = 2020
        df4['year'] = 2019
        df5['year'] = 2018
        df6['year'] = 2017
        df7['year'] = 2016
        df8['year'] = 2015
        # combining dataframes together
        df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])
        # in this case we will not be making a csv file but if you wish to so use the following code to make into a csv file
        df.to_csv(filename, index = False)
        # Return the dataframe to the calling code
        return df

# `Acquire` Data.

In [3]:
df = acquire.get_fifa_data()

/var/folders/dc/8rtfvpyj2mb6mqbnfkf5dl400000gn/T/ipykernel_62628/1900469672.py:1: DtypeWarning: Columns (81,85,104) have mixed types.Specify dtype option on import or set low_memory=False.
  df = acquire.get_fifa_data()


# `Explore` And `Prep` Data

In [4]:
df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2022
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,2022
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,2022
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142074,220806,https://sofifa.com/player/220806/ellis-redman/...,E. Redman,Ellis Redman,CB,41,61,20000.0,2000.0,17,...,41,41,40,10,https://cdn.sofifa.net/players/220/806/15_120.png,https://cdn.sofifa.net/teams/112254/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-wls.png,2015
142075,225509,https://sofifa.com/player/225509/aaron-collins...,A. Collins,Aaron Graham John Collins,ST,41,50,30000.0,2000.0,17,...,31,31,32,14,https://cdn.sofifa.net/players/225/509/15_120.png,https://cdn.sofifa.net/teams/112254/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-wls.png,2015
142076,201197,https://sofifa.com/player/201197/paul-tisdale/...,P. Tisdale,Paul Tisdale,"CM, CAM",40,40,NaN,2000.0,41,...,33,33,34,10,https://cdn.sofifa.net/players/201/197/15_120.png,https://cdn.sofifa.net/teams/143/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/mt.png,2015
142077,217591,https://sofifa.com/player/217591/piotr-zemlo/1...,P. Żemło,Piotr Żemło,"LM, LB",40,50,15000.0,2000.0,18,...,53-3,53-3,51-1,12,https://cdn.sofifa.net/players/217/591/15_120.png,https://cdn.sofifa.net/teams/1873/60.png,https://cdn.sofifa.net/flags/pl.png,NaN,https://cdn.sofifa.net/flags/pl.png,2015


In [5]:
df.columns

Index(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age',
       ...
       'cb', 'rcb', 'rb', 'gk', 'player_face_url', 'club_logo_url',
       'club_flag_url', 'nation_logo_url', 'nation_flag_url', 'year'],
      dtype='object', length=111)

In [6]:
df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2022
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,2022
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,2022
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142074,220806,https://sofifa.com/player/220806/ellis-redman/...,E. Redman,Ellis Redman,CB,41,61,20000.0,2000.0,17,...,41,41,40,10,https://cdn.sofifa.net/players/220/806/15_120.png,https://cdn.sofifa.net/teams/112254/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-wls.png,2015
142075,225509,https://sofifa.com/player/225509/aaron-collins...,A. Collins,Aaron Graham John Collins,ST,41,50,30000.0,2000.0,17,...,31,31,32,14,https://cdn.sofifa.net/players/225/509/15_120.png,https://cdn.sofifa.net/teams/112254/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/gb-wls.png,2015
142076,201197,https://sofifa.com/player/201197/paul-tisdale/...,P. Tisdale,Paul Tisdale,"CM, CAM",40,40,NaN,2000.0,41,...,33,33,34,10,https://cdn.sofifa.net/players/201/197/15_120.png,https://cdn.sofifa.net/teams/143/60.png,https://cdn.sofifa.net/flags/gb-eng.png,NaN,https://cdn.sofifa.net/flags/mt.png,2015
142077,217591,https://sofifa.com/player/217591/piotr-zemlo/1...,P. Żemło,Piotr Żemło,"LM, LB",40,50,15000.0,2000.0,18,...,53-3,53-3,51-1,12,https://cdn.sofifa.net/players/217/591/15_120.png,https://cdn.sofifa.net/teams/1873/60.png,https://cdn.sofifa.net/flags/pl.png,NaN,https://cdn.sofifa.net/flags/pl.png,2015


In [7]:
#add 0 values to non goal keeper players
df['goalkeeping_speed'].fillna("0", inplace = True)

In [8]:
#forces dataframe to show every columns
pd.set_option('display.max_columns', None)
df

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,RW,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96.0,20,35,24,6,11,15,14,8,0,89+3,89+3,89+3,92.0,93,93,93,92.0,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2022
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88.0,35,42,19,15,6,12,8,10,0,90+2,90+2,90+2,85.0,88,88,88,85.0,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,ST,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95.0,24,32,24,7,11,15,14,11,0,90+1,90+1,90+1,88.0,89,89,89,88.0,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://c

# Columns That Have Null Values
- plan:
    adress and acess null values

In [9]:
na = df.isna().sum() > 0
na[na]

value_eur                    True
wage_eur                     True
club_team_id                 True
club_name                    True
league_name                  True
league_level                 True
club_position                True
club_jersey_number           True
club_loaned_from             True
club_joined                  True
club_contract_valid_until    True
nation_team_id               True
nation_position              True
nation_jersey_number         True
release_clause_eur           True
player_tags                  True
player_traits                True
pace                         True
shooting                     True
passing                      True
dribbling                    True
defending                    True
physic                       True
mentality_composure          True
lw                           True
rw                           True
club_logo_url                True
club_flag_url                True
nation_logo_url              True
dtype: bool

# `value_eur`,`wage_eur`, & `club_team_id`.

In [10]:
df[df.value_eur.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

`Takeaways:`
- we might have to consider removing these players because they would corrupt our data considering their wage information is missing and would not add any value to our model.

In [11]:
df[df.wage_eur.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

`Takeaways:`
- we might have to consider removing these players because they would corrupt our data considering their value information is missing and would not add any value to our model.

In [12]:
df[df.club_team_id.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

`Takeaways:`
- we might have to consider removing these players because there club id is important information that we need to conclude their club budget.

In [13]:
df[df.league_name.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

`Takeaways:`
- we might have to consider removing these players because the leage that they play in is important and can give us valuable information that needs to be categorized.

# `club_position`, `club_joined`, & `club_contract_valid_until`.

In [14]:
df[df.club_position.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

In [15]:
df[df.club_joined.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
83,194765,https://sofifa.com/player/194765/antoine-griez...,A. Griezmann,Antoine Griezmann,"ST, LW, RW",85,85,53000000.0,220000.0,30,1991-03-21,176,73,240.0,Atlético de Madrid,Spain Primera Division,1.0,LS,17.0,FC Barcelona,NaN,2022.0,18,France,1335.0,RW,7.0,Left,3,4,4,Medium/Medium,Unique,Yes,NaN,#Acrobat,"Finesse Shot, Flair, Long Shot Taker (AI), Pla...",80.0,84.0,84.0,87.0,52.0,72.0,83,84,83,84,86,85,86,85,82,89,80,80,92,89,83,82,90,86,63,83,73,49,89,85,79,90.0,43,54,49,14,8,14,13,14,0,84+1,84+1,84+1,85.0,86-1,86-1,86-1,85.0,86-1,86-1,86-1,85,82+3,82+3,82+3,85,71+3,69+3,69+3,69+3,71+3,69+3,63+3,63+3,63+3,69+3,21+3,https://cdn.sofifa.net/players/194/765/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1335/60.png,https://cdn.sofifa.net/flags/fr.png,2022
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
163,201153,https://sofifa.com/player/201153/alvaro-borja-...,Morata,Álvaro Borja Morata Martín,ST,83,83,37000000.0,84000.0,28,1992-10-23,190,84,45.0,Juventus,Italian Serie A,1.0,ST,9.0,Atlético de Madrid,NaN,2022.0,45,Spain,1362.0,SUB,7.0,Right,4,3,3,Medium/Medium,Unique,Yes,NaN,NaN,Speed Dribbler (AI),82.0,80.0,72.0,81.0,31.0,77.0,72,84,86,78,80,83,78,44,60,83,79,85,72,80,63,80,84,77,80,72,69,24,87,77,75,79.0,37,14,20,4,5,4,4,5,0,83,83,83,81.0,82,82,82,81.0,80+3,80+3,80+3,79+3,71+3,71+3,71+3,79+3,58+3,55+3,55+3,55+3,58+3,54+3,50+3,50+3,50+3,54+3,13+3,https://cdn.sofifa.net/players/201/153/22_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1362/60.png,https://cdn.sofifa.net/flags/es.png,2022
195,235805,https://sofifa.com/player/235805/federico-chie...,F. Chiesa,Federico Chiesa,"RW, LW, RM",83,91,80500000.0,74000.0,23,1997-10-25,175,70,45.0,Juventus,Italian Serie A,1.0,LM,22.0,Fiorentina,NaN,2022.0,27,Italy,1343.0,RW,14.0,Right,4,4,3,High/Medium,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Acrobat","Long Shot Taker (AI

In [16]:
df[df.club_contract_valid_until.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
147,184087,https://sofifa.com/player/184087/toby-alderwei...,T. Alderweireld,Toby Alderweireld,CB,83,83,NaN,NaN,32,1989-03-02,186,81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,Belgium,1325.0,RCB,2.0,Right,3,2,3,Medium/Medium,Normal (185+),Yes,NaN,NaN,Long Passer (AI),58.0,55.0,70.0,67.0,86.0,77.0,64,45,81,77,38,62,63,59,81,75,55,60,54,85,62,78,81,76,77,58,79,85,52,62,58,86.0,87,87,84,16,6,14,16,14,0,65+3,65+3,65+3,63.0,65,65,65,63.0,66+3,66+3,66+3,66+3,73+3,73+3,73+3,66+3,77+3,81+2,81+2,81+2,77+3,78+3,82+1,82+1,82+1,78+3,21+3,https://cdn.sofifa.net/players/184/087/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png,2022
416,197242,https://sofifa.com/player/197242/grzegorz-kryc...,G. Krychowiak,Grzegorz Krychowiak,"CDM, CM",80,80,NaN,NaN,31,1990-01-29,187,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,Poland,1353.0,CDM,10.0,Right,3,3,3,High/High,Normal (185+),No,NaN,NaN,"Long Passer (AI), Long Shot Taker (AI)",68.0,76.0,72.0,72.0,75.0,81.0,63,72,75,78,75,70,54,62,81,79,67,69,59,80,53,82,74,78,83,78,80,77,72,69,78,80.0,75,76,72,15,14,8,6,7,0,75+3,75+3,75+3,72.0,74,74,74,72.0,74+3,74+3,74+3,73+3,76+3,76+3,76+3,73+3,74+3,78+2,78+2,78+2,74+3,74+3,77+3,77+3,77+3,74+3,18+3,https://cdn.sofifa.net/players/197/242/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png,2022
694,187607,https://sofifa.com/player/187607/artem-dzyuba/...,A. Dzyuba,Artem Dzyuba,ST,78,78,NaN,NaN,32,1988-08-22,196,91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,ST,22.0,Right,3,3,2,Medium/Low,Normal (185+),No,NaN,"#Aerial Threat, #Strength",Leadership,65.0,78.0,69.0,71.0,29.0,86.0,61,81,87,77,74,73,66,60,57,77,63,66,55,75,35,79,65,78,94,69,80,30,81,77,75,79.0,25,17,21,15,12,11,11,8,0,78,78,78,73.0,76,76,76,73.0,74+2,74+2,74+2,73+2,69+2,69+2,69+2,73+2,53+2,54+2,54+2,54+2,53+2,50+2,50+2,50+2,50+2,50+2,18+2,https://cdn.sofifa.net/players/187/607/22_120.png,NaN,NaN,https://cdn.sofifa.net/teams/1357/60.png,https://cdn.sofifa.net/flags/ru.png,2022
926,200094,https://sofifa.com/player/200094/magomed-ozdoe...,M. Ozdoev,Magomed Ozdoev,"CDM, CM",77,78,NaN,NaN,28,1992-11-05,184,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Russia,1357.0,RDM,7.0,Right,3,3,1,Medium/High,Normal (170-185),No,NaN,NaN,Long Passer (AI),75.0,70.0,72.0,76.0,73.0,76.0,65,69,57,78,45,77,58,56,76,76,73,77,80,73,74,81,67,78,75,74,77,75,68,74,54,72.0,73,76,73,11,8,16,16,15,0,71+2,71+2,71+2,73.0,74,74,74,73.0,75+2,75+2,75+2,74+2,75+2,75+2,75+2,74+2,74+2,76+2,76+2,76+2,74+2,73+2,73+2,73+2,73+2,73+2,19+2,https://cdn.sofifa.net/players/200/094/22_120.png,NaN,N

`takeaways:`
- Club_position, club_joined, & club_contract_valid_until all seem to have similar payers with null values.
- We will be dropping these rows to better benefit our exploration and our modeling.

# `nation_team_id`, `nation_position`.

In [17]:
df[df.nation_team_id.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,175,68,73.0,Paris Saint-Germain,French Ligue 1,1.0,LW,10.0,NaN,2017-08-03,2025.0,54,Brazil,NaN,NaN,NaN,Right,5,5,5,High/Medium,Unique,Yes,238700000.0,"#Speedster, #Dribbler, #Playmaker, #FK Special...","Injury Prone, Flair, Speed Dribbler (AI), Play...",91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,93.0,35,32,29,9,9,15,15,11,0,83+3,83+3,83+3,90.0,88,88,88,90.0,89+2,89+2,89+2,89+2,82+3,82+3,82+3,89+2,67+3,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,2022
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
13,200104,https://sofifa.com/player/200104/heung-min-son...,H. Son,손흥민 孙兴慜,"LM, CF, LW",89,89,104000000.0,220000.0,28,1992-07-08,183,78,18.0,Tottenham Hotspur,English Premier League,1.0,LW,7.0,NaN,2015-08-28,2025.0,167,Korea Repu

In [18]:
df[df.nation_position.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,1992-02-05,175,68,73.0,Paris Saint-Germain,French Ligue 1,1.0,LW,10.0,NaN,2017-08-03,2025.0,54,Brazil,NaN,NaN,NaN,Right,5,5,5,High/Medium,Unique,Yes,238700000.0,"#Speedster, #Dribbler, #Playmaker, #FK Special...","Injury Prone, Flair, Speed Dribbler (AI), Play...",91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,93.0,35,32,29,9,9,15,15,11,0,83+3,83+3,83+3,90.0,88,88,88,90.0,89+2,89+2,89+2,89+2,82+3,82+3,82+3,89+2,67+3,63+3,63+3,63+3,67+3,62+3,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png,2022
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
13,200104,https://sofifa.com/player/200104/heung-min-son...,H. Son,손흥민 孙兴慜,"LM, CF, LW",89,89,104000000.0,220000.0,28,1992-07-08,183,78,18.0,Tottenham Hotspur,English Premier League,1.0,LW,7.0,NaN,2015-08-28,2025.0,167,Korea Repu

`Takeaways:`
- Number of Nulls missing is greater than %90 so we will be eliminating these columns.

***

# `pace`, `shooting`,`dribbling`, `passing`, `defending`, `physic`.

In [19]:
df[df.pace.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

In [20]:
df[df.shooting.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

In [21]:
df[df.passing.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

In [22]:
df[df.dribbling.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

In [23]:
df[df.defending.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

In [24]:
df[df.physic.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
5,200389,https://sofifa.com/player/200389/jan-oblak/220002,J. Oblak,Jan Oblak,GK,91,93,112000000.0,130000.0,28,1993-01-07,188,87,240.0,Atlético de Madrid,Spain Primera Division,1.0,GK,13.0,NaN,2014-07-16,2023.0,44,Slovenia,NaN,NaN,NaN,Right,3,1,5,Medium/Medium,Unique,Yes,238000000.0,NaN,"GK Long Throw, Comes For Crosses",NaN,NaN,NaN,NaN,NaN,NaN,13,11,15,43,13,12,13,14,40,30,43,60,67,88,49,59,78,41,78,12,34,19,11,65,11,68.0,27,12,18,87,92,78,90,90,50.0,33+3,33+3,33+3,32.0,35,35,35,32.0,38+3,38+3,38+3,35+3,38+3,38+3,38+3,35+3,32+3,36+3,36+3,36+3,32+3,32+3,33+3,33+3,33+3,32+3,89+3,https://cdn.sofifa.net/players/200/389/22_120.png,https://cdn.sofifa.net/teams/240/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/si.png,2022
7,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Peter Neuer,GK,90,90,13500000.0,86000.0,35,1986-03-27,193,93,21.0,FC Bayern München,German 1. Bundesliga,1.0,GK,1.0,NaN,2011-07-01,2023.0,21,Germany,1337.0,GK,1.0,Right,4,1,5,Medium/Medium,Unique,Yes,22300000.0,NaN,"Leadership, GK Long Throw, Rushes Out Of Goal,...",NaN,NaN,NaN,NaN,NaN,NaN,15,13,25,60,11,30,14,11,68,46,54,60,51,87,35,68,77,43,80,16,29,30,12,70,47,70.0,17,10,11,88,88,91,89,88,56.0,40+3,40+3,40+3,40.0,43,43,43,40.0,47+3,47+3,47+3,44+3,50+3,50+3,50+3,44+3,37+3,43+3,43+3,43+3,37+3,35+3,34+3,34+3,34+3,35+3,88+2,https://cdn.sofifa.net/players/167/495/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1337/60.png,https://cdn.sofifa.net/flags/de.png,2022
8,192448,https://sofifa.com/player/192448/marc-andre-te...,M. ter Stegen,Marc-André ter Stegen,GK,90,92,99000000.0,250000.0,29,1992-04-30,187,85,241.0,FC Barcelona,Spain Primera Division,1.0,GK,1.0,NaN,2014-07-01,2025.0,21,Germany,NaN,NaN,NaN,Right,4,1,4,Medium/Medium,Unique,Yes,210400000.0,NaN,"Rushes Out Of Goal, Comes For Crosses, Saves w...",NaN,NaN,NaN,NaN,NaN,NaN,18,14,11,61,14,21,18,12,63,30,38,50,39,86,43,66,79,35,78,10,43,22,11,70,25,70.0,25,13,10,88,85,88,88,90,43.0,35+3,35+3,35+3,35.0,38,38,38,35.0,42+3,42+3,42+3,39+3,45+3,45+3,45+3,39+3,33+3,41+3,41+3,41+3,33+3,31+3,33+3,33+3,33+3,31+3,88+3,https://cdn.sofifa.net/players/192/448/22_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,NaN,https://cdn.sofifa.net/flags/de.png,2022
12,192119,https://sofifa.com/player/192119/thibaut-court...,T. Courtois,Thibaut Courtois,GK,89,91,85500000.0,250000.0,29,1992-05-11,199,96,243.0,Real Madrid CF,Spain Primera Division,1.0,GK,1.0,NaN,2018-08-09,2026.0,7,Belgium,1325.0,GK,1.0,Left,3,1,4,Medi

`Takeaways:`
- Most of these null values are for goal keepers and can be replaced with 0 because these skills are not a requirement for their career field.

***

In [25]:
df[df.mentality_composure.isnull()]

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url,year
110301,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, CF",94,95,111000000.0,550000.0,28,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,1.0,RW,10.0,NaN,2004-07-01,2018.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",92.0,88.0,86.0,95.0,24.0,62.0,80,93,71,88,85,96,89,90,79,96,95,90,92,92,95,80,68,76,59,88,48,22,90,90,74,NaN,13,23,21,6,11,15,14,8,0,87+3,87+3,87+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,91+3,90+3,82+3,82+3,82+3,90+3,62+3,57+3,57+3,57+3,62+3,57+3,44+3,44+3,44+3,57+3,19+3,https://cdn.sofifa.net/players/158/023/16_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png,2016
110302,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",93,93,85500000.0,475000.0,30,1985-02-05,185,80,243.0,Real Madrid CF,Spain Primera Division,1.0,LM,7.0,NaN,2009-07-01,2018.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",92.0,93.0,80.0,91.0,33.0,78.0,82,95,86,81,87,93,88,77,72,91,91,93,90,92,62,94,94,87,79,93,62,29,93,81,85,NaN,22,31,23,7,11,15,14,11,0,91+2,91+2,91+2,90+3,91+2,91+2,91+2,90+3,88+3,88+3,88+3,88+3,80+3,80+3,80+3,88+3,64+3,60+3,60+3,60+3,64+3,60+3,52+3,52+3,52+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/16_120.png,https://cdn.sofifa.net/teams/243/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png,2016
110303,9014,https://sofifa.com/player/9014/arjen-robben/16...,A. Robben,Arjen Robben,"RM, LM, RW",90,90,56000000.0,250000.0,31,1984-01-23,180,80,21.0,FC Bayern München,German 1. Bundesliga,1.0,SUB,10.0,NaN,2009-08-28,2017.0,34,Netherlands,105035.0,RW,11.0,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",92.0,86.0,82.0,92.0,32.0,64.0,80,85,51,85,86,93,86,83,74,90,92,92,91,91,91,86,61,76,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,0,84+3,84+3,84+3,89+1,88+2,88+2,88+2,89+1,88+2,88+2,88+2,87+3,80+3,80+3,80+3,87+3,65+3,60+3,60+3,60+3,65+3,59+3,47+3,47+3,47+3,59+3,19+3,https://cdn.sofifa.net/players/009/014/16_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1

`Takeaways:`
- Considering that mentality composure has over 30 thousand values missing and there are plenty of other skill sets that we can work with for the players, we will end up dropping this column.

## Drop additional columns

In [26]:
# df = df.drop(columns = ['player_url', 
#                         'long_name', 
#                         'dob', 
#                         'league_level', 
#                         'club_jersey_number', 
#                         'club_loaned_from',
#                         'nation_jersey_number',
#                         'body_type',
#                         'real_face',
#                         'release_clause_eur',
#                         'player_tags', 
#                         'player_traits',
#                         'ls', 'st', 'rs', 'lw', 'lf', 'cf',
#                         'rf', 'rw', 'lam', 'cam', 'ram',
#                         'lm', 'lcm', 'cm', 'rcm', 'rm',
#                         'lwb', 'ldm', 'cdm', 'rdm', 'rwb',
#                         'lb','lcb', 'cb', 'rcb', 'rb',
#                         'gk', 'player_face_url', 'club_logo_url',
#                         'club_flag_url', 'nation_logo_url',
#                         'nation_flag_url','mentality_composure',
#                         'nation_position','nation_team_id',
#                         ])

In [27]:
#add 0 values to non goal keeper players
# df['goalkeeping_speed'].fillna("0", inplace = True)
# df['defending'].fillna("0", inplace = True)
# df['physic'].fillna("0", inplace = True)
# df['dribbling'].fillna("0", inplace = True)
# df['passing'].fillna("0", inplace = True)
# df['shooting'].fillna("0", inplace = True)
# df['shooting'].fillna("0", inplace = True)
# df['pace'].fillna("0", inplace = True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142079 entries, 0 to 142078
Columns: 111 entries, sofifa_id to year
dtypes: float64(16), int64(44), object(51)
memory usage: 120.3+ MB


In [29]:
a = df.isna().sum() > 0
a[a]

value_eur                    True
wage_eur                     True
club_team_id                 True
club_name                    True
league_name                  True
league_level                 True
club_position                True
club_jersey_number           True
club_loaned_from             True
club_joined                  True
club_contract_valid_until    True
nation_team_id               True
nation_position              True
nation_jersey_number         True
release_clause_eur           True
player_tags                  True
player_traits                True
pace                         True
shooting                     True
passing                      True
dribbling                    True
defending                    True
physic                       True
mentality_composure          True
lw                           True
rw                           True
club_logo_url                True
club_flag_url                True
nation_logo_url              True
dtype: bool

In [30]:
#drops na values in colums
#df = df.dropna()

In [31]:
#df

In [32]:
prepare.handle_missing_values(df)

Before dropping nulls, 142079 rows, 111 cols
After dropping nulls. 131874 rows. 64 cols


,sofifa_id,short_name,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,club_name,league_name,club_position,club_joined,club_contract_valid_until,nationality_id,nationality_name,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,year
0,158023,L. Messi,"RW, ST, CF",93,93,78000000.0,320000.0,34,170,72,73.0,Paris Saint-Germain,French Ligue 1,RW,2021-08-10,2023.0,52,Argentina,Left,4,4,5,Medium/Low,85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,20,35,24,6,11,15,14,8,0,2022
1,188545,R. Lewandowski,ST,92,92,119500000.0,270000.0,32,185,81,21.0,FC Bayern München,German 1. Bundesliga,ST,2014-07-01,2023.0,37,Poland,Right,4,4,5,High/Medium,78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,35,42,19,15,6,12,8,10,0,2022
2,20801,Cristiano Ronaldo,"ST, LW",91,91,45000000.0,270000.0,36,187,83,11.0,Manchester United,English Premier League,ST,2021-08-27,2023.0,38,Portugal,Right,4,5,5,High/Low,87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,24,32,24,7,11,15,14,11,0,2022
3,190871,Neymar Jr,"LW, CAM",91,91,129000000.0,270000.0,29,175,68,73.0,Paris Saint-Germain,French Ligue 1,LW,2017-08-03,2025.0,54,Brazil,Right,5,5,5,High/Medium,91.0,83.0,86.0,94.0,37.0,63.0,85,83,63,86,86,95,88,87,81,95,93,89,96,89,84,80,64,81,53,81,63,37,86,90,93,35,32,29,9,9,15,15,11,0,2022
4,192985,K. De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,181,70,10.0,Manchester City,English Premier League,RCM,2015-08-30,2025.0,7,Belgium,Right,5,4,4,High/High,76.0,86.0,93.0,88.0,64.0,78.0,94,82,55,94,82,88,85,83,93,91,76,76,79,91,78,91,63,89,74,91,76,66,88,94,83,68,65,53,15,13,5,10,13,0,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142073,222997,M. Maier,CM,42,54,25000.0,2000.0,18,174,70,111821.0,FC Flyeralarm Admira,Austrian Football Bundesliga,SUB,2013-01-10,2015.0,4,Austria,Right,3,3,1,Medium/Medium,46.0,32.0,42.0,45.0,32.0,47.0,33,32,29,47,20,40,30,22,44,44,47,46,52,52,76,38,56,33,47,32,60,45,35,49,21,31,25,35,15,6,14,10,7,0,2015
142074,220806,E. Redman,CB,41,61,20000.0,2000.0,17,187,81,112254.0,Newport County,English League Two,RES,2013-10-22,2015.0,50,Wales,Right,2,2,1,Medium/Medium,66.0,25.0,29.0,30.0,39.0,53.0,25,25,41,36,25,25,21,25,33,26,65,66,52,36,55,22,62,56,52,25,50,41,25,22,31,38,40,35,12,6,15,8,6,0,2015
142075,225509,A. Collins,ST,41,50,30000.0,2000.0,17,178,57,112254.0,Newport County,English League Two,RES,2014-08-14,2015.0,50,Wales,Right,3,3,1,Medium/Medium,64.0,41.0,27.0,35.0,27.0,41.0,22,47,43,25,27,29,20,22,24,30,60,68,66,39,68,36,51,47,42,36,30,25,41,39,47,25,25,25,13,11,12,15,11,0,2015
142077,217591,P. Żemło,"LM, LB",40,50,15000.0,2000.0,18,190,76,1873.0,Wisła Kraków,Polish T-Mobile Ekstraklasa,RES,2012-07-01,2015.0,37,Poland,Left,3,2,1,Medium/Medium,58.0,27.0,35.0,31.0,52.0,68.0,49,24,57,40,25,27,24,21,34,32,57,58,36,50,44,31,61,74,75,25,44,47,38,21,31,54,52,51,8,14,7,10,9,0,2015
